In [1]:
import numpy
import os
import pandas as pd
from pathlib import Path

In [8]:
path_to_dir_ = Path("/Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/final3")

# # name_ = "viral"
names_ = ["viral", "human", "archea", 
          "bacteria_30", "bacteria_58", "bacteria_88", "bacteria_125"]
# # final_ = "general_results_viral500m.tsv"
finals_ = ["general_results_viral500m.tsv", 
           "general_results_human9g.tsv", 
           "general_results_archea1.4g.tsv",
           "general_results_bacteria30g.tsv",
           "general_results_bacteria58g.tsv",
           "general_results_bacteria88g.tsv",
           "general_results_bacteria125g.tsv",
           "dummy"]
path_finals = [Path("/Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_viral500m.tsv"),
               Path("/Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_human9g.tsv"),
               Path("/Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_archea1.4g.tsv"),
               Path("/Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_bacteria30g.tsv"),
               Path("/Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_bacteria58g.tsv"),
               Path("/Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_bacteria88g.tsv"),
               Path("/Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_bacteria125g.tsv")]
# path_final = Path("/Users/manuez42/Documents/results_fonda/general_results_viral500m.tsv")    

In [9]:
def convert_size(size):
    size = str(size)
    if 'G' in size:
        return float(''.join(ctp(c) for c in size if (c.isdigit() or c == ',' or c == '.'))) * 1024
    elif 'M' in size:
        return float(''.join(ctp(c) for c in size if (c.isdigit() or c == ',' or c == '.')))
    else:
        return float(''.join(ctp(c) for c in size if (c.isdigit() or c == ',' or c == '.')))

def ctp(c):
    if c == ',':
        c = '.'
    return c

def prepare_data(path_to_dir, name, final, path_final):
    filenames = filter(lambda x: (name in x) and (x != final), os.listdir(path_to_dir))
    file_paths = map((lambda x: path_to_dir / x), filenames)
    data = pd.read_csv(path_final, sep='\t')
    pd.set_option('display.max_rows', None)
    # print(data)
    
    for file in file_paths:
    # file = file_paths[0]
    # if file:
        # print(file)
        df = pd.read_csv(file)
        df = df.drop("bin_id", axis=1)
        df = df.dropna(how='any')
        
        df = df.rename(columns={"Data Size in mb": "Data Size"})
        df = df.rename(columns={"Nodes": "Node Name"})
        
        df["Data Size"] = df["Data Size"].apply(convert_size)        
        df["Node Name"] = df["Node Name"].str.replace('cmp', '').astype(int)

        data = pd.concat([data, df], axis=0)
        
    data.to_csv(path_final, index=False, sep='\t')     

In [10]:
# for i in range(0,7):
#     prepare_data(path_to_dir_, names_[i], finals_[i], path_finals[i])

In [19]:
# scp -r manuez42@allegro.imp.fu-berlin.de:/data/scratch/manuez42/A2-job-granularity/MG-HIBF/final2/ /Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/
# scp -r manuez42@allegro.imp.fu-berlin.de:/data/scratch/manuez42/A2-job-granularity/MG-HIBF/final3/ /Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/
# scp -r manuez42@allegro.imp.fu-berlin.de:/data/scratch/manuez42/A2-job-granularity/MG-HIBF/final4/ /Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/


"""
at least 10 per node 

record per nodes
"""




In [11]:
def clean_data(path_to_f, path_to_c, finalname, threshold):
    data = pd.read_csv(path_to_f, sep='\t')
    
    #create stat
    stat = data['Node Name'].value_counts()
    stat = pd.DataFrame({'Node_Name': stat.index, 
                         'number_of_datapoints': stat.values})
    stat['dropped'] = 0
    stat['threshold'] = threshold
    stat['dataset'] = finalname

    stat = stat.sort_values(by='Node_Name')

    #drop unnecessary nodes and update stat
    rows_to_keep = []
    for _, row in stat.iterrows():
        if row['number_of_datapoints'] < threshold:
            stat.loc[stat['Node_Name'] == row['Node_Name'], 'dropped'] = 1
        else:
            rows_to_keep.append(row['Node_Name'])
    
    data = data[data['Node Name'].isin(rows_to_keep)]
    
    # print(rows_to_keep)
    
    data.to_csv(path_to_c/('cleand_' + finalname), index=False, sep='\t')
    
    return stat


In [12]:
stat_file = Path("/Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/clean/stats.tsv")
path_to_clean = Path("/Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/clean")

stat_col = ['dataset', 'Node_Name', 'number_of_datapoints', 'threshold', 'dropped']
stat_overall = pd.DataFrame(columns=stat_col)


threshold = 10

for i in range(0,7):
    stat_single = clean_data(path_finals[i], path_to_clean, finals_[i], threshold)
    stat_overall = pd.concat([stat_overall, stat_single], ignore_index=True)


stat_overall.to_csv(stat_file, index=False, sep='\t')  
    

[213, 215, 216, 217, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 239, 240, 241, 242, 244, 246, 247, 248]
[]
[213, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 240, 241, 242, 244, 246, 247, 248]
[213, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 239, 240, 241, 242, 244, 246, 247, 248]
[213, 215, 216, 217, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 240, 241, 242, 244, 246, 247, 248]
[213, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 239, 240, 241, 242, 244, 246, 247, 248]
[213, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 239, 240, 241, 242, 244, 246, 247, 248]
